In [1]:
import matplotlib.pyplot as plt
import json
with open("mpl_config.json") as fp:
    config: dict = json.load(fp)
for k,v in config.items():
    plt.rcParams[k] = v

# constant velocity

# subsonic case

#  supersonic case

# transonic case